### Load the model
Import necessary libraries:
- `Cobra`: for optimization, FBA and FVA on the genome-scale metabolic model.
- `Networkx`: to generate and analyze a graph of the network.
- `Matplotlib`: for visualization.

In [ ]:
import cobra
import matplotlib.pyplot as plt
import networkx as nx
from cobra import Reaction, Metabolite

Load the genome-scale metabolic model of Synechocystis sp. PCC 6803:

In [ ]:
model = cobra.io.read_sbml_model('..\data\iSynCJ816.xml')

### Set objective function

Set the objective function to be optimized for the model: https://cobrapy.readthedocs.io/en/latest/constraints_objectives.html 

In [ ]:
model.objective = {model.reactions.BIOMASS_Ec_SynAuto_1: 1}

In [ ]:
model.reactions[26]

### Model information
Acces some information about the model:
 - reactions
 - metabolites

In [ ]:
model.reactions.get_by_id('TYRTA')

In [ ]:
model.metabolites[:3]

### FBA
Optimize the model using flux balance analysis:

In [ ]:
solution = model.optimize()
solution

### Model Summary

In [ ]:
model.summary()

### Visualizing flux distribution

In [ ]:
fig, ax = plt.subplots(1,1, figsize = (5, 5))
ax.hist(solution.fluxes.sort_values(), bins=30)
ax.set_yscale('log')
ax.set_xlabel('Flux [1/hr]', fontsize = 12)
ax.set_ylabel('# Reactions (log-scale)', fontsize = 12)
ax.set_title('Histogram of reaction fluxes\nin optimized cyanobacteria network', fontsize = 12)
fig.savefig('figures/CyanoFluxDistribution.jpg')

### Visualizing network graph

In [ ]:
g = nx.Graph()

for reaction in model.reactions:
    for m1 in reaction.reactants:
        for m2 in reaction.products:
            if not m1 in g.nodes:
                g.add_node(m1.name)
            if not m1 in g.nodes:
                g.add_node(m2.name)
            if reaction.upper_bound > 0:
                g.add_edge(m1.name, m2.name)
            if reaction.lower_bound < 0:
                g.add_edge(m2.name, m1.name)

plt.figure(figsize=(8,8))
nx.draw(g, pos=nx.drawing.layout.circular_layout(g),node_size = 1, width = .1)
plt.title('Cyanobacteria metabolic network', fontsize = 20)
plt.savefig('figures/CyanoNetwork.jpg')